In [ ]:
# code required to run on a fresh install or in google colab
root = "/tmp/XCP-explain"
! git clone https://github.com/CPMpy/XCP-explain.git {root}
! cd {root}
! pip install -r {root}/requirements.txt
! pip install cpmpy

# add XCP-explain to the Python path
import sys
if root not in sys.path:
    sys.path.insert(0, root)

# Hands on deductive explanations

In this notebook, we will use another instance of the nurse rostering problem to generate some deductive explanations.

First, let's inspect the instance

In [ ]:
"""
    Some imports used throughout the notebook
"""
import os
import time
from visualize import *

# functions required for generating the model
from read_data import get_data

In [ ]:
from read_data import get_data
from factory import *

instance = os.path.join(root,"Benchmarks/CustomInstance.txt")
data = get_data(instance)
factory = NurseSchedulingFactory(data)

In [ ]:
data.staff[["name", "MaxShifts","MaxWeekends"]]

In [ ]:
print(f"Planning for {data.horizon} days")

Now, let's solve the model.

In the optimization formulation as given by schedulingbenchmarks.org, some constraints or requests may be unsatisfied.

In [ ]:
model, nurse_view = factory.get_optimization_model()
assert model.solve(solver="ortools", num_workers=8) # need 8 workers for efficient solving

print(model.status())
print("Total penalty:", model.objective_value())
visualize(nurse_view.value(), factory)

which requests are not satisfied by this solution?

In [ ]:
requests, _ = factory.shift_on_requests(formulation="hard")

denied_requests = [req for req in requests if req.value() is False]
print("The following requests were denied:")
for req in denied_requests:
    print("-", req)

visualize_constraints(denied_requests, nurse_view, factory, do_clear=False)

In [ ]:
# try it yourself!

# requests, _ = factory.shift_off_requests(formulation="hard")
# cover_constraints, _ = factory.cover(formulation="hard")

# TODO: find out which are not satisfied, and visualize!

Ok, so clearly it's not possible to satisfy all constraints and requests.

But _why_ is that the case? Can we gain more insight in this instance and investigate how the conflic(t)s look like?




We treat all constraints and requests equal, so we get a _decision_ problem

In [ ]:
from cpmpy.tools.explain import mus

model, nurse_view = factory.get_decision_model()

subset = mus(model.constraints)
visualize_constraints(subset, nurse_view, factory)

Are there more MUSes? Of course :-)

Let's enumerate a few of them using the MARCO algorithm

In [ ]:
from cpmpy.tools.explain import marco

model, nurse_view = factory.get_decision_model()

for i, (kind, subset) in enumerate(marco(model.constraints, solver="exact", return_mcs=False)):
    if kind == "MUS":
        display(visualize_constraints(subset, nurse_view, factory))
        
    if i == 0:
        break

Clearly, these MUSes are very different! And some are more interpretable than others.

In the remainder of this notebook, we will influence which MUS is found.

First, by finding prefered MUSes using QuickXplain, then finding optimal MUSes given a cost function

In [ ]:
# QuickXplain first
from cpmpy.tools.explain import quickxplain

model, nurse_view = factory.get_decision_model()

# DIY: craft your own ordering of constraints here!
def get_order(cons):
    if "cover" in str(cons): # Find a MUS that does include many cover constraints
        return 1
    return 10


ordered = sorted(model.constraints, key=get_order)
conflict = quickxplain(ordered)
for c in conflict:
    print("-", c)

In [ ]:
visualize_constraints(conflict, nurse_view, factory)

#### Optimal MUS

Now find truely OPTIMAL MUSes given a cost function $f$


In [ ]:
## Careful, this takes a while if you are not using Exact!
from cpmpy.tools.explain import optimal_mus

model, nurse_view = factory.get_decision_model()

# DIY: craft your own cost for constraints here!
def get_weight(cons):
    return 1 # find the smallest one

solver = "exact" if "exact" in cp.SolverLookup.solvernames() else "ortools"
hs_solver = "gurobi" if "gurobi" in cp.SolverLookup.solvernames() else "ortools"

conflict = optimal_mus(model.constraints, 
                       weights=[get_weight(c) for c in model.constraints],
                       solver=solver,
                       hs_solver=hs_solver)
print(f"Found conflict of size {len(conflict)}")

In [ ]:
visualize_constraints(conflict, nurse_view, factory)

# Part 2, fixing UNSAT models

Now that we know _why_ a model is UNSAT, we need to fix it.

In the presentation, several techniques are shown for doing so.

Below, you can find some skeleton code to play around with feasibiliy restoration techniques

In [ ]:
model, nurse_view = factory.get_decision_model()
model.solve()

In [ ]:
from cpmpy.tools.explain import mss_opt, mcs_opt

# DIY: craft your own cost for constraints here!
def get_weight(cons):
    return 1 # equal weights

# find Max-CSP solution
optimal_subset = mss_opt(model.constraints, hard=[],weights=[get_weight(c) for c in model.constraints])
mcs = set(model.constraints) - set(optimal_subset)
print("Found solution after dropping these constraints:")
for i,c in enumerate(mcs):
    print(f"{i}.", c)


In [ ]:
assert cp.Model(optimal_subset).solve() is True
visualize(nurse_view.value(), factory)
visualize_constraints(mcs, nurse_view, factory, do_clear=False)

### Slack-based relaxation

Apart from dropping constraints, they can also be _relaxed_ when numeric

In [ ]:
model, nurse_view, slack_under, slack_over = factory.get_slack_model()  # CMPpy Model

# minimize the maximal violation
slack = cp.cpm_array(np.append(slack_under, slack_over))
model.minimize(cp.max(slack))

assert model.solve()

visualize(nurse_view.value(), factory)

In [ ]:
# DIY: craft your own objective functions